<a href="https://colab.research.google.com/github/GeonKimdcu/DeepLearning/blob/main/Chapter04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 04 최적화
---

## 4.1 확률적 경사 하강법

확률적 경사 하강법은 손실 함수의 곡면에서 **'경사가 가장 가파른 곳으로 내려가다 보면 언젠가 가장 낮은 지점에 도달한다'**는 가정으로 만들어졌다. <br>
알고리즘의 가정이 단순한 만큼 다양한 상황에 잘 대처하지 못하고 학습 속도도 느리기 때문에 성능에 한계가 있다. 확률적 경사 하강법에 어떤 문제점이 있고 어떤 방향으로 개선되었는지 살펴보자.

### 4.1.1 고정된 학습률

**학습률**이란 최적화할 때 **한걸음의 폭**을 결정하는 <font color='#ff6f61'>**스텝 크기**</font>를 말하며 **학습 속도를 결정**한다. <br>
확률적 경사 하강법은 지정된 학습률을 사용하는 알고리즘이므로 경험적으로 학습률을 조정할 수밖에 없다.

#### 학습률을 조정하지 않으면 무슨 일이 생길까?

학습률이 고정되어 있기 때문에 최적화가 비효율적으로 진행된다. **학습률이 낮으면 학습 속도가 느리고**, 반대로 **학습률이 높으면 최적해로 수렴하지 못하거나 심지어는 손실이 점점 커지는 방향으로 발산**하기도 한다.

<img width=600 src=https://user-images.githubusercontent.com/48666867/147644287-5220edd9-1245-40f9-a5e4-a3b6ecb301da.png>

Ref. https://ichi.pro/ko/silje-hagseublyul-iljeong-keras-mich-tensorflow-2-0eul-sayonghan-ye-51989279443468

#### 학습률 감소

일반적으로 **학습 초기**에는 <font color='#ff6f61'>큰 폭으로 이동해서 최대한 빠르게 내려가고</font>, **어느 정도 내려가면** <font color='#ff6f61'>작은 폭으로 천천히 이동해서</font> 최적해에 조심스럽게 접근하는 것이 좋다. <br>
그래서 <font color='#ff6f61'>처음에는 **높은 학습률**로 학습을 시작하고 학습이 진행되는 상황에 따라 학습률을 조금씩 낮추는데 이런 방식을 **학습률 감소**(learning rate decay)</font>라고 한다.

#### 적응적 학습률

<font color='#ff6f61'>**경사가 가파를 때**</font> **큰 폭으로 이동하면 최적화 경로를 벗어나거나 최소 지점을 지나칠 수 있으므로 적은 폭으로 천천히 이동**하는 것이 좋고, <font color='#ff6f61'>**경사가 완만**</font>하면 **큰 폭으로 빠르게 이동**하는 것이 좋다.

따라서 **곡면의 변화**에 따라 **학습률이 적응적으로 조정된다면 안정적으로 학습**할 수 있다. <br>
이와 같이 <font color='#ff6f61'>학습의 진행 상황이나 곡면의 변화를 알고리즘 내에서 고려해서 학습률을 자동으로 조정하는 방식을 **적응적 학습률**(adaptive learning rate)</font>이라고 한다.

### 4.1.2 안장점에서 학습 종료

손실 함수 곡면에서 **임계점**(critical point)은 미분값이 0인 지점으로 **최대점**(maximum point), **최소점**(minimum point), **안장점**(saddle point)이 된다. <br>

안장점은 한쪽 축에서 보면 최소인데 다른 축에서 보면 최대인 지점으로 말의 안장과 같이 생겼다고 해서 붙여진 이름이다. <br>
**확률적 경사 하강법은 임계점을 만나면 최대점, 최소점, 안장점을 구분하지 못하고 학습을 종료**한다.

문제는 손실 함수 곡면에는 안장점이 무수히 많다는 점이다. 따라서 안장점을 만나더라도 학습이 계속 진행될 방법이 필요하다.

<img width=350 src=https://user-images.githubusercontent.com/48666867/147645269-3d724ae4-47d9-4f26-91bb-bb499aa1923b.png>

Ref. wikipedia

#### 손실 함수 곡면에는 안장점이 얼마나 많을까?

$n$차원 공간에서 임계점을 만났을 때 차원별로 최대일 확률이 $1\over 2$이고 최소일 확률이 $1\over 2$라고 가정해 보자. <br>
임계점이 최소점일 확률은 모든 차원에서 최소이어야 하므로 $\frac{1}{2^n}$이고, 같은 논리로 지역 최대점일 확률도 $\frac {1}{2^n}$이다. <br>
나머지 경우는 모두 안장점이기 때문에 임계점이 안장점일 확률은 $P(안장점)=1-\frac{1}{2^{n-1}}$가 되어 <font color='#ff6f61'>**차원이 높아질수록 안장점일 확률은 점점 높아진다.**</font>

#### 임계점을 만났을 때 최대점 $\centerdot$ 최소점 $\centerdot$ 안장점을 어떻게 구분할까?

임계점에서 **손실**(loss)을 확인해보면 최대점 $\centerdot$ 최소점 $\centerdot$ 안장점을 어느정도 구분할 수 있다. 손실이 매우 낮다면 최소점일 가능성이 크고 손실이 매우 높다면 최대점일 가능성이 크다. 반면 손실이 수렴했다고 보기에 매우 낮지 않으면 안장점이라고 추정할 수 있다.

<img width=400 src=https://user-images.githubusercontent.com/48666867/147646292-0487852f-c8a5-4d20-b1d3-120cfef9886d.png>


### 4.1.3 학습 경로의 진동

확률적 경사 하강법에서 **미니배치 단위로 그레디언트를 근사**하면 원래의 손실 함수보다 **거친 표면을 갖는 손실 함수를 근사하는 셈**이다. <br>
따라서 **기울기가 자주 바뀌는 거친 표면에서 이동하므로 최적화 경로가 진동**한다.

## 4.2 SGD 모멘텀

**SGD 모멘텀**(momentum)은 최적해를 향해 진행하던 속도에 <font color='#ff6f61'>관성</font>을 주어 SGD의 느린 학습 속도와 협곡과 안장점을 만났을 때 학습이 안되는 문제, 거친 표면에서 진동하는 문제를 해결한 최적화 알고리즘이다.

### 4.2.1 핵심 개념

SGD가 가장 가파른 곳으로 내려가는 방식이라면 SGD 모멘텀은 **지금까지 진행하던 속도에 관성이 작용**하도록 만든 방식이다. <br>

다음 그림에서 **그레디언트 벡터**는 현재 위치에서 가장 가파른 방향이고 **속도 벡터**는 현재 이동하는 속도를 나타낸다. <br>
SGD 모멘텀은 <font color='#ff6f61'>현재의 속도 벡터와 그레디언트 벡터를 더해서 다음 위치를 정한다.</font>

<img width=350 src=https://user-images.githubusercontent.com/48666867/147656495-25cf43f9-e397-4cf9-b262-fff026d23e51.png>

이렇게 **속도에 관성이 작용하면 지금가지 진행하던 방향과 다른 방향에 내리막길이 나타나더라도 갑자기 방향을 바꾸지 않는다.** <br>
관성이 작용하면서 **학습 경로가 전체적으로 매끄러워지고 가파른 경사를 만나면 가속도가 생겨서 학습이 매우 빨라진다.**

SGD 모멘텀을 수식으로 표현하면 다음과 같다. 여기서 $v_t$는 현재 속도이고 $\rho$는 **마찰 계수**(friction coefficient)로서 보통 0.9나 0.99를 사용한다.

$$v_{t+1}=\rho v_t + \triangledown f(x_t) \quad \rho : 마찰 계수, \;v_t: 현재 속도, \;\triangledown f(x_t): 현재\;그레디언트\\x_{t+1}=x_t-\alpha v_{t+1}$$ 

다음 속도 $v_{t+1}$은 현재 속도에 마찰 계수를 곱한 뒤 그레디언트를 더해서 계산한다. <br>
파라미터 업데이트 식은 SGD의 파라미터 업데이트 식과 같은 형태이며 그레디언트 대신 속도를 사용한다.

### 4.2.2 관성을 이용한 임계점 탈출과 빠른 학습

관성이 생기면 진행하던 속도로 계속 진행하려고 하므로, 안장점을 만나거나 깊이가 얕은 지역 최소에 빠지더라도 그 지점을 벗어날 수 있다. <br>
손실 함수의 표면이 울퉁불퉁하면 기울기가 계속 바뀌기 때문에 다른 쪽으로 튕겨 나갈 수 있는데, 이때 관성이 작용하면 진행하던 속도를 유지하며 부드럽게 이동할 수 있다.

<img width=300 src=https://user-images.githubusercontent.com/48666867/147660932-8ee7f2b5-eb31-4f0e-9df3-dd7e441452e6.png>

Ref. http://www.yaldex.com

경로를 결정할 때 SGD는 기울기만 적용했고 SGD 모멘텀은 기울기에 속도의 관성을 적용했기 때문에 SGD 모멘텀이 더 빠르게 수렴한다.

<img width=300 src=https://user-images.githubusercontent.com/48666867/147661214-af15c2b5-128e-4346-a1e1-b306bba15e85.png>

<img width=300 src=https://user-images.githubusercontent.com/48666867/147661252-fb9c5630-54ed-4bc6-a06a-dc01ccac6de6.png>

Ref. https://thushv89.medium.com/


### 4.2.3 오버 슈팅 문제

SGD 모멘텀은 <font color='#ff6f61'>**오버 슈팅**</font>(overshooting)이 되는 단점이 있다. <br>
경사가 가파르면 **빠른 속도로 내려오다가 최소 지점을 만나면 그레디언트는 순간적으로 작아지지만 속도는 여전히 크기 대문에 최소 지점을 지나쳐 오버 슈팅**이 된다. <br>
아래 그림은 최적화 경로가 최소 지점을 지나쳐서 반대편으로 높이 올라간 상황이다.

<img width=350 src=https://user-images.githubusercontent.com/48666867/147661311-1d7566cf-bec1-4537-b6a8-01ef26103ed5.png>

Ref. https://thushv89.medium.com/

이를 벡터로 표현하면 아래 그림과 같다. <br>
그레디언트 벡터는 작지만 속도 벡터가 크기 때문에 실제 이동 스텝도 커져서 오버 슈팅이 된다.

<img width=250 src=https://user-images.githubusercontent.com/48666867/147661637-f85d1e96-f401-45c1-be5c-80b5807a872e.png>

**최소 지점이 평평한 평지 위에 있다면 그레디언트와 속도가 모두 작으므로 오버 슈팅이 되지 않는다.** <br>
하지만 최소 지점 주변의 경사가 가파르다면 속도가 크기 때문에 오버 슈팅이 될 수밖에 없다. 이때 최적해 주변을 평평하게 만들어주는 **정규화 기법**을 사용하면 오버 슈팅을 막을 수 있다.

SGD 모멘텀은 다음과 같은 식으로 표현할 수 있다.

$$v_{t+1}=\rho v_t-\alpha \triangledown f(x_t)\\x_{t+1}=x_t+v_{t+1}$$


## 4.3 네스테로프 모멘텀

**네스테로프 모멘텀**(Nesterov momentum)은 진행하던 속도에 관성을 준다는 점은 SGD 모멘텀과 같지만 오버 슈팅을 막기 위해 <font color='#ff6f61'>현재 속도로 한 걸음 미리 가보고</font> 오버 슈팅이 된 만큼 <font color='#ff6f61'>다시 내리막길로 내려가기 </font>때문에 이동 방향에 차이가 있다.

### 4.3.1 핵심 개념

오버 슈팅이 생기지 않도록 한 걸음 미리 갔을 때 높이 올라간 만큼 다시 내려오도록 그레디언트로 교정해 준다. <br>

이를 그림으로 설명하면 **속도 벡터**는 현재 이동하는 속도를 나타내고 **그레디언트 벡터**는 현재 속도로 한 걸음 미리 가 본 위치에서 내리막길 방향을 의미한다.

<img width=250 src=https://user-images.githubusercontent.com/48666867/147666612-57add85b-af23-4fac-9d9e-097733d05258.png>

따라서 네스테로프 모멘텀은 현재의 속도 벡터와 현재 속도로 한 걸음 미리 가본 위치의 그레디언트 벡터를 더해서 다음 위치를 정한다.

네스테로프 모멘텀을 수식으로 표현하면 다음과 같다. 여기서 $v_t$는 현재 속도이고 $\rho$는 마찰 계수로 0.9나 0.99를 사용한다.

$$v_{t+1}=\rho v_t - \alpha \triangledown f(x_t+\rho v_t) \quad ,(x_t+\rho v_t): 한\;걸음\;미리\;가기\\x_{t+1}=x_t+v_{t+1}$$

다음 속도 $v_{t+1}$은 현재 속도에 마찰 계수를 곱한 뒤 현재 속도로 한 걸음 미리 가 본 위치의 그레디언트를 빼서 계산한다.

### 4.3.2 오버 슈팅 억제

네스테로프 모멘텀과 SGD 모멘텀을 비교해보면 정말 오버 슈팅을 잘 억제하는지 아래의 그림을 통해 알 수 있다.

<img width=300 src=https://user-images.githubusercontent.com/48666867/147667014-cf825d49-2b8a-456d-88aa-89d0e86f3b5e.png>
<img width=300 src=https://user-images.githubusercontent.com/48666867/147666887-d38833f5-6fad-46a9-9fe7-3eb3196602b7.png>

Ref. https://thushv89.medium.com

## 4.4 AdaGrad

**AdaGrad**(adaptive gradient)는 <font color='#ff6f61'>손실 함수의 곡면의 변화에 따라 적응적으로 학습률을 정하는 알고리즘</font>이다.

### 4.4.1 핵심 개념

최적의 학습률을 정하기 위해선 곡면의 변화량에 학습률이 적응적으로 조정되어야 한다.

<img width=500 src=https://user-images.githubusercontent.com/48666867/147667900-b691857f-a329-4860-b68a-f13760d2dc81.png>

왼쪽 그림은 변화가 크면 적은 폭으로 이동하는 하는 모습이고, 오른쪽 그림은 변화가 없으면 큰 폭으로 이동하는 모습이다.

곡면의 변화량은 어떻게 계산할까? <br>
**곡면의 변화가 크다는 것은 기울기의 변화가 크다**는 의미이므로, **모든 단계에서 계산했던 기울기를 모아서 크기를 측정**해보면 된다.

먼저 모든 단계의 기울기를 하나의 벡터로 표현해 보자.

<img width=300 src=https://user-images.githubusercontent.com/48666867/147668328-b8958dd6-57c1-465a-baca-f37a250ddacf.png>

$$기울기 벡터 = (\triangledown f(x_1), \triangledown f(x_2), ..., \triangledown f(x_t)$$

<font color='#ff6f61'>각 기울기의 제곱합을 계산해서 곡면의 변화량</font>으로 사용할 수 있다.

$$곡면의 변화량 = r_{t+1}=\triangledown f(x_1)^2+\triangledown f(x_2)^2+...+\triangledown f(x_t)^2$$

SGD의 파라미터 업데이트 식에서 학습률 $\alpha$를 곡면의 변화량의 제곱근 $\sqrt{r_{t+1}}$로 나눠주면 **적응적 학습률**이 된다. <br>
적응적 학습률은 <font color='#ff6f61'>곡면의 변화량에 반비례</font>하므로, 곡면의 변화가 크면 천천히 학습하고 곡면에 변화가 작으면 빠르게 학습한다.

다음 수식은 적응적 학습률이 반영된 파라미터 업데이트 식으로 '그레디언트의 제곱을 누적하는 식'과 '파라미터 업데이트 식'으로 정의된다.

<img width=400 src=https://user-images.githubusercontent.com/48666867/147669565-50a2db91-541e-4f1a-861d-9ee91da62fba.jpg>

Ref. Deep learning textbook

---
✨$\epsilon$은 분모가 0이 되지 않게 더해주는 아주 작은 상수로 보통 $1e-7$이나 $1e-8$을 사용한다.

<br>

AdaGrad는 **모델의 파라미터별로 곡면의 변화량을 계산**하기 때문에 <font color='#ff6f61'>파라미터별로 개별 학습률을 갖는 효과</font>가 생긴다. <br>
**손실 함수에서 파라미터의 차원별로 곡면의 변화량이 다를 수 있기 때문에 파라미터별로 개별 학습률을 가지면 좀 더 정확하고 빠르게 최적해로 수렴**할 수 있다.



### 4.4.2 학습 초반에 학습이 중단되는 현상

AdaGrad는 한 가지 치명적인 문제가 있다. <br>
곡면의 변화량을 전체 경로의 기울기 벡터의 크기로 계산하므로 **학습이 진행될수록 곡면의 변화량은 점점 커지고 반대로 적응적 학습률은 점점 낮아진다는 점**이다. <br>
만일 <font color='#ff6f61'>경사가 매우 가파른 위치에서 학습을 시작</font>한다면, <font color='#ff6f61'>초반부터 적응적 학습률이 급격히 감소</font>하기 시작해서 <font color='#ff6f61'>조기에 학습이 중단</font>될 수 있다.

<img width=350 src=https://user-images.githubusercontent.com/48666867/147669673-d89b4562-222a-4bdb-a524-cfe82dd2b615.jpg>

Ref. Deep learning textbook

## 4.5 RMSProp

**RMSProp**(root mean square propagation)은 <font color='#ff6f61'>**최근 경로의 곡면 변화량**에 따라 학습률을 적응적으로 결정하는 알고리즘</font>이다.

AdaGrad가 조기에 학습이 중단되는 문제를 해결하기 위해 RMSProp은 곡면 변화량을 개선된 방식으로 측정한다.

### 4.5.1 핵심 개념

곡면 변화량을 측정할 때 전체 경로가 아닌 **최근 경로의 변화량**을 측정하면 곡면 변화량이 누적되어 계속해서 증가하는 현상을 없앨 수 있다.

<img width=600 src=https://user-images.githubusercontent.com/48666867/147670057-12905124-961b-40c8-ab97-05e29e135128.png>

Ref. velog.io/@qsdcfd/

RMSProp은 최근 경로의 곡면 변화량을 측정하기 위해 <font color='#ff6f61'>지수가중이동평균</font>(exponentially weighted moving average)을 사용한다. <br>

다음 수식을 보면 **$r_{t+1}$를 계산할 때 $r_t$와 그레디언트의 제곱 $\triangledown f(x_t)^2$을 가중 합산해서 지수가중이동평균을 계산**했다.

<img width=600 src=https://user-images.githubusercontent.com/48666867/147670598-7c0c109c-0b21-4126-b0ac-4ac2a9cbe458.png>

Ref. velog.io/@qsdcfd/

---
✨$\beta$는 가중치로 보통 0.9를 사용하며, 𝜖 은 분모가 0이 되지 않게 더해주는 아주 작은 상수로 보통  1𝑒−7 이나  1𝑒−8 을 사용한다.

### 4.5.2 최근 경로의 곡면 변화량

RMSProp의 첫 번째 식인 $r_{t+1}$에 대한 점화식을 풀어보자. <br>
먼저 $r_t=\beta r_{t-1}+(1-\beta)\triangledown f(x_{t-1})^2$를 첫 번재 식에 대입한다.

$$r_{t+1}=\beta r_t+(1-\beta)\triangledown f(x_t)^2\\=\beta(\beta r_{t-1}+(1-\beta)\triangledown f(x_{t-1})^2)+(1-\beta)\triangledown f(x_t)^2\\=\beta^2r_{t-1}+\beta(1-\beta)\triangledown f(x_{t-1})^2+(1-\beta)\triangledown f(x_t)^2\\=\beta^2r_{t-1}+(1-\beta)(\triangledown f(x_t)^2+\beta \triangledown f(x_{t-1})^2)$$

같은 방식으로 $r_{t-1}$부터 $r_1$까지 순서대로 대입하면 다음과 같은 식이 정리된다.

$$r_{t+1}=\beta^tr_1+(1-\beta)(\triangledown f(x_t)^2+\beta \triangledown f(x_{t-1})^2 +\dotsb+\beta^{t-1}\triangledown f(x_1)^2)$$

---
✨$\triangledown f(x_t)^2$: 최근 변화는 많이 반영됨, $\beta^{t-1}\triangledown f(x_1)^2$: 오래된 변화는 적게 반영됨.

<br><br>

이 식의 괄호 안을 보면 $x_1$ 지점에서 첫 번째 그레디언트 제곱 $\triangledown f(x_1)^2$의 가중치는 $\beta^{t-1}$이고, $x_2$ 지점에서 두 번째 그레디언트 제곱 $\triangledown f(x_2)^2$의 가중치는 $\beta^{t-2}$이다. <br>
<font color='#ff6f61'>현재 시점에서 멀어질수록 가중치가</font> $1,\beta^1,...,\beta^{t-2},\beta^{t-1}$와 같이 <font color='#ff6f61'>$\beta=0.9$의 지수승으로 변화</font>하면서 점점 <font color='#ff6f61'>0으로 수렴</font>한다. <br>

따라서 <font color='#ff6f61'>최근 경로의 그레디언트는 많이 반영</font>되고 <font color='#ff6f61'>오래된 경로의 그레디언트는 작게 반영</font>된다.

그레디언트 제곱에 곱해지는 가중치가 지수승으로 변화하기 때문에 지수가중평균이라고 부르며, 단계마다 새로운 그레디언트 제곱의 비율을 반영하여 평균이 업데이트되기 때문에 지수가중이동평균이라고 부른다.

## 4.6 Adam

**Adam**(adaptive moment estimation)은 **SGD 모멘텀과 RMSProp이 결합**된 형태로, 진행하던 속도에 <font color='#ff6f61'>관성</font>을 주고 동시에 최근 경로의 곡면의 변화량에 따라 <font color='#ff6f61'>적응적 학습률</font>을 갖는 알고리즘이다.

### 4.6.1 핵심 개념

Adam은 관성에 대한 장점과 적응적 학습률에 대한 장점을 모두 갖는 전략을 취한다. <br>
그래서 **최적화 성능이 우수**하고 **잡음 데이터에 대해 민감하게 반응하지 않는 성질**이 있다. 그리고 두 알고리즘을 합치면서 <font color='#ff6f61'>학습 초기 경로가 편향되는 RMSProp의 문제를 제거</font>했다.

Adam을 식으로 표현하면 다음 수식과 같다. <br>
첫 번째 식은 **1차 관성**으로서 <font color='#ff6f61'>속도</font>를 계산한다. SGD 모멘텀의 첫 번째 식에 해당하는데, 속도에 마찰 계수 $\rho$ 대신 가중치 $\beta_1$을 곱해서 그레디언트의 지수가중이동평균을 구하는 형태로 수정되었다. 

두 번째 식은 **2차 관성**으로서 <font color='#ff6f61'>그레디언트 제곱의 지수가중이동평균</font>을 구하는 식이다. RMSProp의 첫 번째 식과 동일하다. 

세 번째 식은 파라미터 업데이트 식으로 1차 관성과 2차 관성을 사용한다.

$$v_{t+1}=\beta_1v_t+(1-\beta_1)\triangledown f(x_t)\\r_{t+1}=\beta_2r_t+(1-\beta_2)\triangledown f(x_t)^2\\x_{t+1}=x_t-\frac{\alpha}{\sqrt{r_{t+1}}+\epsilon} \odot v_{t+1}$$

---
✨가중치 $\beta_1$과 $\beta_2$는 보통 0.9나 0.99를 사용하며 $\epsilon$은 분모가 0이 되지 않게 더해주는 상수로 보통 $1e-7$이나 $1e-8$를 사용한다.

✨$\frac{\alpha}{\sqrt{r_{t+1}}+\epsilon}$은 최근 경로의 곡면의 변화량을 뜻하고, $v_{t+1}$은 관성을 갖는 속도를 뜻한다.


### 4.6.2 학습 초기 경로 편향 문제

위와 같은 식으로 학습하면 첫 번째 단계에서 출발 지점으로부터 멀리 떨어진 곳으로 이동하는 초기 경로의 편향 문제가 생긴다.

#### 초기 경로에 편향이 발생하는 이유

<img width=300 src=https://user-images.githubusercontent.com/48666867/147675044-0ecd8add-48d6-4f8f-9a0e-524d4047c618.jpg>

Ref. Deep learning textbook

훈련을 시작할 때는 1차 관성 $v_0$과 2차 관성 $r_0$ 모두 0이다. 이 상태에서 첫 번째 단계를 실행하면 $v_1=0.1*\triangledown f(x_0),\;r_1=0.01*\triangledown f(x_0)^2$이 된다. <br>
이 때 $r_1$의 식에 있는 0.01이 값이 작기 때문에 $r_1$도 작아질 수 있다. $r_1$이 작아지면 **적응적 학습률이 커져서 출발 지점에서 멀리 떨어진 곳으로 이동하게 되고 운이 나쁘면 최적화에 좋지 않은 곳으로 갈 수도 있다.**

#### 초기 경로의 편향 제거

<img width=400 src=https://user-images.githubusercontent.com/48666867/147675509-56567bfc-334d-4fe9-b47d-10652822c12a.png>

Adam은 이 편향을 제거하기 위해 위와 같은 수식을 적용해서 다음과 같이 알고리즘을 개선했다.

<img width=500 src=https://user-images.githubusercontent.com/48666867/147675516-10595196-6384-450d-8f59-2680d65420c4.png>

Ref. velog.io/@qsdcfd/

첫 번째 단계에서 $v_1=0.1\triangledown f(x_0), \; r_1=0.01\triangledown f(x_0)^2$이 계산되면 추가된 식에서 0.1과 0.01을 상쇄하고 $v_1=\triangledown f(x_0),\; r_1=\triangledown f(x_0)^2$이 된다.

따라서 $r_1$은 그레디언트의 제곱이므로 아주 작아질 일은 없다. <br>
이로써 학습 초반에 학습률이 급격히 커지는 편향이 제거되고, 훈련이 진행될수록 $\beta_1$과 $\beta_2$가 1보다 작기 때문에 두 식의 분모 $(1-\beta_1^t)$와 $(1-\beta_2^t)$는 1에 수렴하므로 원래 알고리즘으로 바뀐다.

## Summary

1. <font color='#ff9900'>확률적 경사 하강법</font>은 손실 함수의 곡면에서 '경사가 가장 가파른 곳으로 내려가다 보면 언젠가 가장 낮은 지점에 도달한다'라는 단순한 가정으로 만들어진 만큼 성능에 한계가 있다.
2. 확률적 경사 하강법은 지정된 <font color='#ff9900'>학습률</font>을 사용하는 알고리즘이므로 경험적으로 <font color='#ff9900'>학습률</font>을 조정해야 한다. <font color='#ff9900'>학습률</font>을 작게 설정하면 학습이 느리게 진행되며 <font color='#ff9900'>학습률</font>을 크게 설정하면 최적해로 수렴하지 못할 수 있다.
3. SGD가 가장 가파른 곳으로 내려가는 방식이라면 <font color='#ff9900'>SGD 모멘텀</font>은 지금까지 진행하던 속도에 관성을 주면서 내려가는 방식이다.
4. 관성이 작용하면 <font color='#ff9900'>안장점</font>을 만나거나 깊이가 얕은 지역 최소에 빠지더라도 벗어날 수 있으며, 손실 함수의 표면이 울퉁불퉁하더라도 진행하던 속도를 유지하며 부드럽게 이동하면서 학습 경로가 전체적으로 매끄러워진다.
5. <font color='#ff9900'>네스테로프 모멘텀</font>은 진행하던 속도에 관성을 준다는 점은 SGD 모멘텀과 같지만, 오버 슈팅을 막기 위해 현재 속도로 한 걸음 미리 가보고 오버 슈팅이 된 만큼 다시 내리막길로 내려가는 방식이다.
6. 네스테로프 모멘텀은 관성이 커지더라도 <font color='#ff9900'>오버 슈팅</font>이 될지 미리 살펴보고 교정하기 때문에 <font color='#ff9900'>오버 슈팅</font>이 억제된다.
7. <font color='#ff9900'>AdaGrad</font>는 손실 함수의 곡면 변화에 따라 적응적으로 학습률을 정하는 알고리즘이다.
8. AdaGrad는 학습이 진행될수록 <font color='#ff9900'>곡면 변화량</font>은 점점 증가하고 <font color='#ff9900'>적응적 학습률</font>은 점점 낮아진다. 만일 경사가 가파른 곳에서 출발할 경우 학습 초반부터 <font color='#ff9900'>적응적 학습률</font>이 급격히 감소해서 학습이 조기 중단될 수 있다.
9. <font color='#ff9900'>RMSProp</font>은 곡면 변화량을 측정할 때 전체 경로가 아닌 최근 경로의 곡면 변화량을 측정하여 AdaGrad에서 발생하는 조기에 학습이 중단되는 현상을 없앴다.
10. RMSProp은 최근 경로의 곡면 변화량을 측정하기 위해 <font color='#ff9900'>지수가중이동평균</font> 방식으로 곡면 기울기의 제곱을 반영한다. <font color='#ff9900'>지수가중이동평균</font>을 하면 최근 경로의 그레디언트는 많이 반영하고 오래된 경로의 그레디언트는 작게 반영하므로 기울기의 크기가 점점 누적되어 곡면의 변화량이 커지는 현상이 사라진다.
11. <font color='#ff9900'>Adam</font>은 SGD 모멘텀과 같이 진행하던 속도에 관성을 주고 RMSProp과 같이 학습률을 적응적으로 조정하는 알고리즘이다.
12. <font color='#ff9900'>RMSProp</font>은 출발 지점에서 멀리 떨어진 곳으로 이동하는 초기 경로의 편향 문제가 있는데, Adam은 초기 경로의 편향을 제거하였다.

아래의 그림은 SGD 및 SGD의 변형 알고리즘들이 최적값을 찾는 과정을 시각화 한 것이다.

<img width=400 src=https://user-images.githubusercontent.com/48666867/147676911-45c94da2-78c9-4b8e-8420-00a2b806d7cc.gif>

Ref. https://shuuki4.github.io/

# Reference

1. Do it! 딥러닝 교과서

---

*made by* <font color='6699ff'>Kim Geon</font>(DCU, major.AI·BD)

---